# Advent of Code 2022

## Day 9: Rope Bridge

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

I used a generator again to read the input. I yield each step of the movement as a unit-length displacement

e.g. `U 3` yields

    (1, 0)
    (1, 0)
    (1, 0)

The purpose was to be able to simplify the main loop to process one step at a time.

The choice of Up = positive first vector component is arbitrary, but initially I was going to use a Numpy array to track where the tail had been and render the output for testing, and this would have mapped more directly to that (with the array flipped upside down for rendering, e could keep the start as 0, 0). But it turned out to be easier to use a `set` to track the tail position and then read its length, and we don't really need absolute positive or orientation information anyway, everything can be relative.

I think the `main` loop turned out quite tidy. The only difference between parts 1 and 2 are that head, tail(each a 2-tuple of ints) becomes a list of 10 2-tuples, we move the heed then loop over 1 .. 9 to apply the tail movement.

Most of the bugs I encountered in this puzzle was in the code for calculating movement, that section could use more refactoring.

### Imports

In [ ]:
from typing import Optional, Iterator, Union
from math import sqrt

### Input Reading

In [ ]:
VELOCITY = {
    'U': (1, 0),  # setting up to be positive first component (arbitrary)
    'D': (-1, 0),
    'R': (0, 1),  # setting right to be positive second component
    'L': (0, -1)
}

def read_lines(filename: str, limit: Optional[int] = None) -> Iterator[tuple[int, int]]:
    count = 0
    with open(filename) as file:
        for line in file:
            line = line.strip()
            left, right = line.split(' ')
            for _ in range(int(right)):
                yield VELOCITY[left]
            count += 1
            if limit is not None and count >= limit:
                break

In [ ]:
INPUT_FILE = 'data/input09.txt'

### Code for Calculating Movement

In [ ]:
def move(a: tuple[int, int], da: tuple[int, int]) -> tuple[int, int]:
    return a[0] + da[0], a[1] + da[1]

def norm(x: int) -> int:
    assert type(x) == int
    return int(x / abs(x))

def distance(a: tuple[int, int], b: tuple[int, int]) -> Union[int, float]:
    d0 = abs(a[0] - b[0])
    d1 = abs(a[1] - b[1])
    if not d0 and not d1:
        return 0
    if not d0:
        return int(d1)
    if not d1:
        return int(d0)
    rv = sqrt(d0 ** 2 + d1 ** 2)
    return float(rv)

def step_tail_orthogonal(tail: tuple[int, int], head: tuple[int, int]) -> tuple[int, int]:
    d0 = int(head[0] - tail[0])
    d1 = int(head[1] - tail[1])
    if d0:
        if abs(d0) > 1:
            return norm(d0), 0
        return 0, 0
    if abs(d1) > 1:
        return 0, norm(d1)
    return 0, 0

def step_tail_diagonal(tail: tuple[int, int], head: tuple[int, int]) -> tuple[int, int]:
    d0 = int(head[0] - tail[0])
    d1 = int(head[1] - tail[1])
    if distance(tail, head) > sqrt(2):
        return norm(d0), norm(d1)
    return 0, 0

def step_tail(tail: tuple[int, int], head: tuple[int, int]) -> tuple[int, int]:
    if head == tail:
        return 0, 0
    d = distance(tail, head)
    if type(d) is int:
        rv = step_tail_orthogonal(tail, head)
        return rv
    rv = step_tail_diagonal(tail, head)
    return rv

### Part 1

In [ ]:
def main():

    head, tail, history = (0, 0), (0, 0), {(0, 0)}

    for step in read_lines(INPUT_FILE):

        # move the head
        head = move(head, step)

        # move the tail afterwards, in response to head move
        tail = move(tail, step_tail(tail, head))
        history.add(tail)

        assert distance(head, tail) in {0, 1, sqrt(2)}

    print(f'The tail visited {len(history)} points.')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
def main():

    # initial state
    rope, history = [(0, 0)] * 10, {(0, 0)}

    for step in read_lines(INPUT_FILE):

        # move the head
        rope[0] = move(rope[0], step)

        for t in range(1, len(rope)):
            h = t - 1

            # move the next link in response to the previous link moving
            step_t = step_tail(rope[t], rope[h])
            rope[t] = move(rope[t], step_t)

            assert distance(rope[h], rope[t]) in {0, 1, sqrt(2)}

        history.add(rope[-1])

    print(f'The tail visited {len(history)} points.')

In [ ]:
if __name__ == '__main__':
    main()